In [34]:
import DatasetPrepare
import pandas as pd

In [35]:
# get Envi dataset
df = pd.DataFrame(DatasetPrepare.getEnviDatasets())
df = df.insert(column=)

,Envi
Date,
2015-01-04,364302.0
2015-01-11,330411.0
2015-01-18,372237.0
2015-01-25,393408.0
2015-02-01,553199.0
...,...
2018-02-25,257878.0
2018-03-04,272245.0
2018-03-11,320199.0
